In [45]:
import requests # Contains methods used to make HTTP requests
import xml.etree.ElementTree as ET # Contains methods used to build and parse XML
import sys
import re
import getpass
import os
import conf as config


from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext

from requests.packages.urllib3.fields import RequestField
from requests.packages.urllib3.filepost import encode_multipart_formdata
import tableauserverclient as TSC

password = config.conf["password"]
server_url = config.conf["server"]
site_name = config.conf["site"]
username = config.conf["username"]
archive_project = config.conf["project"]
version  = config.conf["API_version"]

In [46]:
# The namespace for the REST API is 'http://tableausoftware.com/api' for Tableau Server 9.0
# or 'http://tableau.com/api' for Tableau Server 9.1 or later
xmlns = {'t': 'http://tableau.com/api'}

# The maximum size of a file that can be published in a single request is 64MB
FILESIZE_LIMIT = 1024 * 1024 * 64   # 64MB

# For when a workbook is over 64MB, break it into 5MB(standard chunk size) chunks
CHUNK_SIZE = 1024 * 1024 * 5    # 5MB

# If using python version 3.x, 'raw_input()' is changed to 'input()'
raw_input=input


class ApiCallError(Exception):
    pass

class UserDefinedFieldError(Exception):
    pass


In [47]:
def _check_status(server_response, success_code):
    """
    Checks the server response for possible errors.
    'server_response'       the response received from the server
    'success_code'          the expected success code for the response
    Throws an ApiCallError exception if the API call fails.
    """
    if server_response.status_code != success_code:
        parsed_response = ET.fromstring(server_response.text)

        # Obtain the 3 xml tags from the response: error, summary, and detail tags
        error_element = parsed_response.find('t:error', namespaces=xmlns)
        summary_element = parsed_response.find('.//t:summary', namespaces=xmlns)
        detail_element = parsed_response.find('.//t:detail', namespaces=xmlns)

        # Retrieve the error code, summary, and detail if the response contains them
        code = error_element.get('code', 'unknown') if error_element is not None else 'unknown code'
        summary = summary_element.text if summary_element is not None else 'unknown summary'
        detail = detail_element.text if detail_element is not None else 'unknown detail'
        error_message = '{0}: {1} - {2}'.format(code, summary, detail)
        raise ApiCallError(error_message)
    return

def _encode_for_display(text):
    """
    Encodes strings so they can display as ASCII in a Windows terminal window.
    This function also encodes strings for processing by xml.etree.ElementTree functions.
    Returns an ASCII-encoded version of the text.
    Unicode characters are converted to ASCII placeholders (for example, "?").
    """
    return text.encode('ascii', errors="backslashreplace").decode('utf-8')


def sign_in(server, username, version, password, site=""):
    """
    Signs in to the server specified with the given credentials
    'server'   specified server address
    'username' is the name (not ID) of the user to sign in as.
               Note that most of the functions in this example require that the user
               have server administrator permissions.
    'password' is the password for the user.
    'site'     is the ID (as a string) of the site on the server to sign in to. The
               default is "", which signs in to the default site.
    Returns the authentication token and the site ID.
    """
    url = server + "/api/{0}/auth/signin".format(version)

    # Builds the request
    xml_request = ET.Element('tsRequest')
    credentials_element = ET.SubElement(xml_request, 'credentials', name=username, password=password)
    ET.SubElement(credentials_element, 'site', contentUrl=site)
    xml_request = ET.tostring(xml_request)

    # Make the request to server
    server_response = requests.post(url, data=xml_request)
    _check_status(server_response, 200)

    # ASCII encode server response to enable displaying to console
    server_response = _encode_for_display(server_response.text)

    # Reads and parses the response
    parsed_response = ET.fromstring(server_response)

    # Gets the auth token and site ID
    token = parsed_response.find('t:credentials', namespaces=xmlns).get('token')
    site_id = parsed_response.find('.//t:site', namespaces=xmlns).get('id')
    user_id = parsed_response.find('.//t:user', namespaces=xmlns).get('id')
    return token, site_id, user_id


def get_workbook_id(server, auth_token, user_id, site_id, workbook_name):
    """
    Gets the id of the desired workbook to relocate.
    'server'        specified server address
    'auth_token'    authentication token that grants user access to API calls
    'user_id'       ID of user with access to workbook
    'site_id'       ID of the site that the user is signed into
    'workbook_name' name of workbook to get ID of
    Returns the workbook id and the project id that contains the workbook.
    """
    url = server + "/api/{0}/sites/{1}/users/{2}/workbooks".format(VERSION, site_id, user_id)
    server_response = requests.get(url, headers={'x-tableau-auth': auth_token})
    _check_status(server_response, 200)
    xml_response = ET.fromstring(_encode_for_display(server_response.text))

    workbooks = xml_response.findall('.//t:workbook', namespaces=xmlns)
    for workbook in workbooks:
        if workbook.get('name') == workbook_name:
            return workbook.get('id')
    error = "Workbook named '{0}' not found.".format(workbook_name)
    raise LookupError(error)


def sign_out(server, auth_token, version):
    """
    Destroys the active session and invalidates authentication token.
    'server'        specified server address
    'auth_token'    authentication token that grants user access to API calls
    """
    url = server + "/api/{0}/auth/signout".format(version)
    server_response = requests.post(url, headers={'x-tableau-auth': auth_token})
    _check_status(server_response, 204)
    return


def download(server, auth_token, site_id, workbook_id, version):
    """
    Downloads the desired workbook from the server (temp-file).
    'server'        specified server address
    'auth_token'    authentication token that grants user access to API calls
    'site_id'       ID of the site that the user is signed into
    'workbook_id'   ID of the workbook to download
    Returns the filename of the workbook downloaded.
    """
    print("\tDownloading workbook to a temp file")
    url = server + "/api/{0}/sites/{1}/workbooks/{2}/content".format(version, site_id, workbook_id)
    server_response = requests.get(url, headers={'x-tableau-auth': auth_token})
    _check_status(server_response, 200)

    # Header format: Content-Disposition: name="tableau_workbook"; filename="workbook-filename"
    filename = re.findall(r'filename="(.*)"', server_response.headers['Content-Disposition'])[0]
    with open(filename, 'wb') as f:
        f.write(server_response.content)
    return filename

In [3]:
def delete_server_file(server_url, site_name, username, password, workbook_name, archive_project):
    
    tableau_auth = TSC.TableauAuth(username, password, site_name)
    server = TSC.Server(server_url, use_server_version=True)
    with server.auth.sign_in(tableau_auth):

        try:
            dest_project = server.projects.filter(name=archive_project)[0]
        except IndexError:
            raise LookupError(f"No project named {archive_project} found.")

        for wb in TSC.Pager(server.workbooks):

            if (wb.name == workbook_name) and (wb.project_id == dest_project.id): 

                try:
                    server.workbooks.delete(wb.id)
                except IndexError:
                    raise LookupError(f"No workbook named {wb.id} found")

In [24]:
def active_workbooks():
    
    logger = logging.getLogger()
    
    conn = None
    try:
        conn = psycopg2.connect(host="tableau.lendingworks.co.uk",
                                port=8060,
                                database="workgroup",
                                user="readonly",
                                password="HWWwZGmFCN7ikoN98hirG9kBbBVHpr66yHNfBqAmY6oW9DXkYy")

        curr = conn.cursor()

        query = "\
            SELECT\
                last_view_time,\
                views_workbook_id\
            FROM\
                public._views_stats\
            WHERE\
                last_view_time > CURRENT_DATE - INTERVAL '3 months';"

        curr.execute(query)
        conn.commit()

        df = pd.DataFrame(curr.fetchall(), columns=[desc[0] for desc in curr.description])
        active_workbooks = df["views_workbook_id"].astype(str).to_list()

    except Exception as error: 
        logger.error(f"Error while connecting to server repositry. {error}")
        logger.error(error, exc_info=True)

    finally:
        if conn:  
            curr.close()
            conn.close()

    return active_workbooks

In [30]:
import psycopg2
import logging
import pandas as pd
import datetime
from datetime import timedelta 
from dateutil.relativedelta import relativedelta


def workbook_classifier():
    
    logger = logging.getLogger()
    
    conn = None
    try:
        conn = psycopg2.connect(host="tableau.lendingworks.co.uk",
                                port=8060,
                                database="workgroup",
                                user="readonly",
                                password="HWWwZGmFCN7ikoN98hirG9kBbBVHpr66yHNfBqAmY6oW9DXkYy")

        curr = conn.cursor()

        query = "\
            SELECT\
                v.last_view_time,\
                v.views_workbook_id,\
                u.name\
            FROM\
                public._views_stats v\
            JOIN\
                public._users u\
            ON\
                u.id = v.users_id\
            WHERE\
                last_view_time > CURRENT_DATE - INTERVAL '4 months';"


        curr.execute(query)
        conn.commit()

        df = pd.DataFrame(curr.fetchall(), columns=[desc[0] for desc in curr.description])


        now = datetime.datetime.utcnow()
        active_limit = now - relativedelta(months=3)
        reminder_limit = now - relativedelta(months=3) - relativedelta(weeks=1)
        delete_limit = now - relativedelta(months=3) - relativedelta(weeks=2)

        active_workbooks = df[(df["last_view_time"]>active_limit)]["views_workbook_id"].astype(str).to_list()
        reminder_workbooks = df[(df["last_view_time"]>reminder_limit)&(df["last_view_time"]<active_limit)]["views_workbook_id"].astype(str).to_list()
        to_be_deleted_workbooks = df[(df["last_view_time"]>delete_limit)&(df["last_view_time"]<reminder_limit)]["views_workbook_id"].astype(str).to_list()

        reminder_workbooks = list(set(reminder_workbooks) - set(active_workbooks))
        to_be_deleted_workbooks = list(set(to_be_deleted_workbooks) - set(active_workbooks))

        return list(set(active_workbooks)), reminder_workbooks, to_be_deleted_workbooks




    except Exception as error: 
        logger.error(f"Error while connecting to server repositry. {error}")
        logger.error(error, exc_info=True)

    finally:
        if conn:  
            curr.close()
            conn.close()









In [ ]:
query = "\
    SELECT\
        d.server,\
        d.has_extract,\
        w.name,\
        s.name,\
        p.name\
    FROM\
        public.data_connections d\
    JOIN\
        public.workbooks w\
    ON\
        w.id = d.owner_id\
    JOIN\
        public.projects p\
    ON\
        w.project_id = p.id\
    JOIN\
        public.users u\
    ON\
        u.id = w.owner_id\
    JOIN\
        public.system_users s\
    ON\
        S.id = U.system_user_id;"

query = "\
    SELECT\
        last_view_time,\
        views_workbook_id\,\
        users_id\
    FROM\
        public._views_stats\
    WHERE\
        last_view_time > CURRENT_DATE - INTERVAL '3 months';"



In [2]:
def delete_loacl_file(filename):

    if os.path.exists("./{}".format(filename)):
        os.remove("./{}".format(filename))
    else:
        print("The file does not exist!")

In [1]:
"""first check a directory exists, if not, make the directory"""

def get_sharepoint_context_using_user(username, password, sharepoint_url):
 
    # Get sharepoint credentials
    url = sharepoint_url

    # Initialize the client credentials
    user_credentials = UserCredential(username, password)

    # create client context object
    ctx = ClientContext(url).with_credentials(user_credentials)

    return ctx


def upload_to_sharepoint(dir_name: str, file_name: str, username, password):

    relative_url = f'Shared Documents/{dir_name}'
    ctx = get_sharepoint_context_using_user(username, password)

    target_folder = ctx.web.get_folder_by_server_relative_url(relative_url)

    with open(file_name, 'rb') as content_file:
        file_content = content_file.read()
        target_folder.upload_file(file_name, file_content).execute_query()


In [ ]:
def download_and_delete_wokrbooks(username, password, site_name, server_url, workbook_names, filename, SP_dir, SP_username, SP_password):

    ##### STEP 1: Sign in to the server using the API #####
    source_auth_token, source_site_id, source_user_id = sign_in(server_url, username, version, password)

    for workbook_name in workbook_names:
        
        ##### STEP 4: Find workbook id (using the API) #####
        workbook_id = get_workbook_id(server_url, source_auth_token, source_user_id, source_site_id, workbook_name)

        ##### STEP 5: Download workbook (using the API) #####
        workbook_filename = download(server_url, source_auth_token, source_site_id, workbook_id, version)

        ##### STEP 5: Upload file on sharepoint #####
        upload_to_sharepoint(SP_dir, workbook_filename, SP_username, SP_password)

        ##### STEP 5: Delete local file #####
        delete_loacl_file(filename)

        ##### STEP 5: Delete file from server #####
        delete_server_file(server_url, site_name, username, password, workbook_name)

        ##### STEP 6: Sign out of the server (using the API)#####
        sign_out(server_url, source_auth_token, version)


In [58]:
import psycopg2
import logging
import pandas as pd
import datetime
from datetime import timedelta 
from dateutil.relativedelta import relativedelta


def wokrbook_events():
    
    logger = logging.getLogger()
    
    conn = None
    try:
        conn = psycopg2.connect(host="tableau.lendingworks.co.uk",
                                port=8060,
                                database="workgroup",
                                user="readonly",
                                password="HWWwZGmFCN7ikoN98hirG9kBbBVHpr66yHNfBqAmY6oW9DXkYy")

        curr = conn.cursor()

        query = "\
            SELECT\
                *\
            FROM\
                public.workbooks \
            WHERE\
                updated_at > CURRENT_DATE - INTERVAL '4 months';"


        curr.execute(query)
        conn.commit()

        df = pd.DataFrame(curr.fetchall(), columns=[desc[0] for desc in curr.description])


        return df




    except Exception as error: 
        logger.error(f"Error while connecting to server repositry. {error}")
        logger.error(error, exc_info=True)

    finally:
        if conn:  
            curr.close()
            conn.close()

df = wokrbook_events()


In [59]:
df

,id,name,repository_url,description,created_at,updated_at,owner_id,project_id,view_count,size,...,last_published_at,data_id,reduced_data_id,published_all_sheets,extract_encryption_state,extract_creation_pending,is_deleted,parent_workbook_id,is_private,modified_by_user_id
0,1387,May FOV,MayFOV,archive score FOV RiCE,2021-06-17 11:49:29.262,2022-07-28 11:07:17.394,54,113,2,59445801,...,2021-06-17 13:37:03.833,None,1812077b-5002-42d9-a9aa-10ac2df37c1c,True,0.0,0.0,False,None,None,54
1,1692,Jan Quotes Avg APR,JanQuotesAvgAPR,Ad hoc request for the APR in Jan-22.,2022-01-25 16:32:00.623,2022-05-27 13:15:48.742,59,61,2,5313944,...,2022-01-25 16:32:00.620,None,258bd147-58e5-4980-a4c2-ea90e81f3a42,True,NaN,NaN,False,None,None,59
2,1098,Lender Capital Contribution Validation,LenderCapitalContributionValidation,,2020-12-22 10:34:25.542,2022-08-03 10:48:39.711,20,113,2,30435,...,2020-12-22 10:34:25.535,0b26b279-01a2-4a7c-8fe8-fe92a4d393d8,None,True,NaN,NaN,False,None,None,20
3,1754,Scale Up Recalibration - Current Econ,ScaleUpRecalibration-CurrentEcon,Scale Up Test Workbook,2022-03-29 15:24:01.328,2022-05-23 15:26:19.764,54,48,9,3524779,...,2022-04-11 09:51:51.826,None,7c7178fd-ebed-4927-ba4c-63dbf31f104d,False,NaN,NaN,False,None,None,54
4,1153,Starling Accounts,StarlingAccounts,,2021-02-04 09:46:47.534,2022-07-28 11:07:18.028,20,113,1,204907,...,2021-04-09 15:39:24.913,None,46ef8f07-cc67-4dd9-95b4-db2a63a5631d,True,0.0,0.0,False,None,None,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,99,Margin and Profitability Analysis,MarginandProfitabilityAnalysis,,2016-12-02 11:47:56.913,2022-07-28 11:05:47.239,20,113,11,60167221,...,2020-01-10 17:23:04.221,None,ce9db2ac-5005-49b5-9f8c-c42af6e067ad,False,0.0,NaN,False,None,None,20
630,1332,May Score GK Test Data,MayScoreGKTestData,archived score cutoff tests,2021-05-20 12:55:06.924,2022-07-28 11:05:47.407,54,113,1,68990,...,2021-05-20 12:55:06.921,737a0670-6d8c-4619-9a06-80ee2a51e05f,None,True,NaN,NaN,False,None,None,54
631,997,Employment Industry - AN-2228,EmploymentIndustry-AN-2228,,2020-09-23 13:33:20.313,2022-07-28 11:05:47.638,20,113,6,19102190,...,2020-09-23 13:33:20.307,None,2b7e28c1-6a0a-4bcf-8b51-ee1f547c5341,True,NaN,NaN,False,None,None,20
632,1476,loan_requests,loan_requests,,2021-07-30 09:00:48.114,2022-07-28 11:05:47.796,20,113,4,158072,...,2021-07-30 09:20:09.036,3ee63293-7a12-4dd3-b13b-ee160bf8fa40,None,True,NaN,NaN,False,None,None,20


In [51]:
def workbook_classifier():
    
    logger = logging.getLogger()
    
    conn = None
    try:
        conn = psycopg2.connect(host="tableau.lendingworks.co.uk",
                                port=8060,
                                database="workgroup",
                                user="readonly",
                                password="HWWwZGmFCN7ikoN98hirG9kBbBVHpr66yHNfBqAmY6oW9DXkYy")

        curr = conn.cursor()

        query = "\
            SELECT\
                v.last_view_time,\
                v.views_workbook_id,\
                u.name,\
                w.workbook_url\
            FROM\
                public._views_stats v\
            JOIN\
                public._users u\
            ON\
                u.id = v.users_id\
            JOIN\
                public._workbooks w\
            ON\
                w.id = v.views_workbook_id\
            WHERE\
                last_view_time > CURRENT_DATE - INTERVAL '4 months';"


        curr.execute(query)
        conn.commit()

        df = pd.DataFrame(curr.fetchall(), columns=[desc[0] for desc in curr.description])


        now = datetime.datetime.utcnow()
        active_limit = now - relativedelta(months=3)
        reminder_limit = now - relativedelta(months=3) + relativedelta(weeks=1)

        active_workbooks = df[(df["last_view_time"]>active_limit)]["views_workbook_id"].astype(str).to_list()
        reminder_workbooks = df[(df["last_view_time"]<reminder_limit)&(df["last_view_time"]>active_limit)][["views_workbook_id","name","workbook_url"]].drop_duplicates()
        to_be_archived_workbooks = df[(df["last_view_time"]<active_limit)][["views_workbook_id","name","workbook_url"]].drop_duplicates()
        
        reminder_workbooks = reminder_workbooks.drop_duplicates()
        to_be_archived_workbooks = to_be_archived_workbooks.drop_duplicates()

        return list(set(active_workbooks)), reminder_workbooks, to_be_archived_workbooks

        
    except Exception as error: 
        logger.error(f"Error while connecting to server repositry. {error}")
        logger.error(error, exc_info=True)

    finally:
        if conn:  
            curr.close()
            conn.close()

In [52]:
to_be_deleted_workbooks, reminder_workbooks_owner, to_be_deleted_workbooks_owner = workbook_classifier()

In [55]:
reminder_workbooks_owner[reminder_workbooks_owner["views_workbook_id"]==1494]

,views_workbook_id,name,workbook_url
193,1494,david.marshall@lendingworks.co.uk,ConversionLast1monthsdailydeduping2-hourlyrefr...
790,1494,Graham.Dodds@lendingworks.co.uk,ConversionLast1monthsdailydeduping2-hourlyrefr...
817,1494,oliver.relph@lendingworks.co.uk,ConversionLast1monthsdailydeduping2-hourlyrefr...
852,1494,Stephanie.McArdle@lendingworks.co.uk,ConversionLast1monthsdailydeduping2-hourlyrefr...
928,1494,ines.maia@lendingworks.co.uk,ConversionLast1monthsdailydeduping2-hourlyrefr...


In [28]:
now = datetime.datetime.utcnow()
active_limit = now - relativedelta(months=3)
reminder_limit = now - relativedelta(months=3) - relativedelta(weeks=1)
delete_limit = now - relativedelta(months=3) - relativedelta(weeks=2)

to_be_deleted_workbooks = df[(df["last_view_time"]>delete_limit)&(df["last_view_time"]<reminder_limit)]["views_workbook_id"].astype(str).to_list()
to_be_deleted_workbooks_names = df[(df["last_view_time"]>delete_limit)&(df["last_view_time"]<reminder_limit)]["name"].astype(str).to_list()

In [33]:
list(zip(to_be_deleted_workbooks,to_be_deleted_workbooks_names))

[('79', 'richard@lendingworks.co.uk'),
 ('23', 'tableau.admin@lendingworks.co.uk'),
 ('1174', 'oliver.relph@lendingworks.co.uk'),
 ('1017', 'millie.parry@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1634', 'richard@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1577', 'richard@lendingworks.co.uk'),
 ('1494', 'Adam.Robery@lendingworks.co.uk'),
 ('1494', 'Adam.Robery@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1579', 'Adam.Robery@lendingworks.co.uk'),
 ('1684', 'Stephanie.McArdle@lendingworks.co.uk'),
 ('1234', 'millie.parry@lendingworks.co.uk'),
 ('1006', 'virginie.allaire@lendingworks.co.uk'),
 ('541', 'arthur.cox@lendingworks.co.uk'),
 ('1522', 'J